In [1]:
import os
import cv2
import numpy as np

def yolo_poly_to_pixels(img_w, img_h, poly):
    """
    Convert normalized YOLO polygon to pixel coordinates.
    poly: list of [x1, y1, x2, y2, ..., xn, yn] (normalized)
    """
    return [(int(poly[i] * img_w), int(poly[i + 1] * img_h)) for i in range(0, len(poly), 2)]

def draw_segmentation(image, label_path, color_map, alpha=0.5):
    h, w = image.shape[:2]
    if not os.path.exists(label_path):
        return image

    # Create an overlay for transparency
    overlay = image.copy()

    with open(label_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        parts = list(map(float, line.strip().split()))
        if not parts:
            continue
        class_id = int(parts[0])
        poly_norm = parts[1:]
        poly_pixel = yolo_poly_to_pixels(w, h, poly_norm)

        pts = np.array([poly_pixel], np.int32)
        color = color_map.get(class_id, (0, 255, 0))  # Default green

        # Draw filled polygon on overlay
        cv2.fillPoly(overlay, [pts], color=color)

    # Blend overlay with original image
    cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0, image)
    return image

def create_image_grid(images, filenames, grid_size=None, padding=10, label_height=30, font_scale=0.5):
    num_images = len(images)
    if grid_size is None:
        grid_w = grid_h = int(np.ceil(np.sqrt(num_images)))
    else:
        grid_w, grid_h = grid_size

    if num_images == 0:
        raise ValueError("No images provided to create a grid.")

    h, w = images[0].shape[:2]

    # Add space for label below each image
    grid_img = np.zeros((
        grid_h * h + padding * (grid_h + 1) + label_height * grid_h,
        grid_w * w + padding * (grid_w + 1), 3), dtype=np.uint8)

    for idx, img in enumerate(images):
        if idx >= grid_w * grid_h:
            break
        row = idx // grid_w
        col = idx % grid_w

        start_y = row * h + padding * (row + 1) + row * label_height
        start_x = col * w + padding * (col + 1)

        # Place image
        grid_img[start_y:start_y+h, start_x:start_x+w, :] = img

        # Put filename label below image
        label_bg_top = start_y + h
        label_bg_bottom = label_bg_top + label_height
        cv2.rectangle(grid_img, (start_x, label_bg_top), (start_x + w, label_bg_bottom),
                      (40, 40, 40), -1)  # Dark background

        text_size, _ = cv2.getTextSize(filenames[idx], cv2.FONT_HERSHEY_SIMPLEX, font_scale, 1)
        text_x = start_x + (w - text_size[0]) // 2
        text_y = label_bg_top + (label_height + text_size[1]) // 2

        cv2.putText(grid_img, filenames[idx], (text_x, text_y),
                    cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), 1, cv2.LINE_AA)

    return grid_img

def visualize_segmentation_dataset_flat(image_dir, label_dir, output_path=None,
                                       target_size=(640, 640), grid_size=None):
    # Get all image files
    image_files = [
        f for f in os.listdir(image_dir)
        if f.lower().endswith(('.png', '.jpg', '.jpeg'))
    ]
    image_files.sort()

    if not image_files:
        print("No valid image files found.")
        return

    # First pass: collect class IDs to assign colors
    class_ids = set()
    for img_file in image_files:
        base_name = os.path.splitext(img_file)[0]
        label_file = os.path.join(label_dir, base_name + ".txt")
        if not os.path.exists(label_file):
            continue
        with open(label_file, 'r') as f:
            for line in f:
                if line.strip():
                    class_id = int(line.strip().split()[0])
                    class_ids.add(class_id)

    # Assign unique color per class
    color_map = {}
    np.random.seed(0)
    for cid in sorted(class_ids):
        color_map[cid] = tuple(map(int, np.random.randint(0, 255, size=3)))

    # Second pass: load, resize, annotate
    annotated_images = []
    image_filenames = []

    for img_file in image_files:
        base_name = os.path.splitext(img_file)[0]
        label_file = os.path.join(label_dir, base_name + ".txt")
        img_path = os.path.join(image_dir, img_file)

        img = cv2.imread(img_path)
        if img is None:
            print(f"Failed to read image: {img_path}")
            continue

        # Resize image
        resized_img = cv2.resize(img, target_size)
        # Draw annotations
        annotated_img = draw_segmentation(resized_img, label_file, color_map, alpha=0.8)
        annotated_images.append(annotated_img)
        image_filenames.append(img_file)

    # Create grid
    try:
        grid_image = create_image_grid(annotated_images, image_filenames, grid_size=grid_size)
    except Exception as e:
        print(f"Error creating grid: {e}")
        return

    # Save or show result
    if output_path:
        cv2.imwrite(output_path, grid_image)
        print(f"Saved visualization to {output_path}")
    else:
        cv2.imshow("Segmentation Dataset Overview", grid_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [3]:
image_dir = r'C:\Users\ochon\OneDrive\Documents\2025\National AI Competition\dataset\train\images'
label_dir = r'C:\Users\ochon\OneDrive\Documents\2025\National AI Competition\dataset\train\labels'
output_image = r'C:\Users\ochon\OneDrive\Documents\2025\National AI Competition\dataset\train-viz.jpg'

visualize_segmentation_dataset_flat(
    image_dir=image_dir,
    label_dir=label_dir,
    output_path=output_image,
    target_size=(640, 640),  # You can change this as needed
    grid_size=None  # Automatically calculated based on sqrt(n)
)

Saved visualization to C:\Users\ochon\OneDrive\Documents\2025\National AI Competition\dataset\train-viz.jpg


In [5]:
image_dir = r'C:\Users\ochon\OneDrive\Documents\2025\National AI Competition\dataset\val\images'
label_dir = r'C:\Users\ochon\OneDrive\Documents\2025\National AI Competition\dataset\val\labels'
output_image = r'C:\Users\ochon\OneDrive\Documents\2025\National AI Competition\dataset\val-viz.png'

visualize_segmentation_dataset_flat(
    image_dir=image_dir,
    label_dir=label_dir,
    output_path=output_image,
    target_size=(640, 640),  # You can change this as needed
    grid_size=None  # Automatically calculated based on sqrt(n)
)

Saved visualization to C:\Users\ochon\OneDrive\Documents\2025\National AI Competition\dataset\val-viz.webp
